# Day 16: Packet Decoder

[https://adventofcode.com/2021/day/16](https://adventofcode.com/2021/day/16)

## Description

### Part One

As you leave the cave and reach open waters, you receive a transmission from the Elves back on the ship.

The transmission was sent using the Buoyancy Interchange Transmission System (<span title="Just be glad it wasn't sent using the BuoyancY Transmission Encoding System.">BITS</span>), a method of packing numeric expressions into a binary sequence. Your submarine's computer has saved the transmission in [hexadecimal](https://en.wikipedia.org/wiki/Hexadecimal) (your puzzle input).

The first step of decoding the message is to convert the hexadecimal representation into binary. Each character of hexadecimal corresponds to four bits of binary data:

    0 = 0000
    1 = 0001
    2 = 0010
    3 = 0011
    4 = 0100
    5 = 0101
    6 = 0110
    7 = 0111
    8 = 1000
    9 = 1001
    A = 1010
    B = 1011
    C = 1100
    D = 1101
    E = 1110
    F = 1111
    

The BITS transmission contains a single _packet_ at its outermost layer which itself contains many other packets. The hexadecimal representation of this packet might encode a few extra `0` bits at the end; these are not part of the transmission and should be ignored.

Every packet begins with a standard header: the first three bits encode the packet _version_, and the next three bits encode the packet _type ID_. These two values are numbers; all numbers encoded in any packet are represented as binary with the most significant bit first. For example, a version encoded as the binary sequence `100` represents the number `4`.

Packets with type ID `4` represent a _literal value_. Literal value packets encode a single binary number. To do this, the binary number is padded with leading zeroes until its length is a multiple of four bits, and then it is broken into groups of four bits. Each group is prefixed by a `1` bit except the last group, which is prefixed by a `0` bit. These groups of five bits immediately follow the packet header. For example, the hexadecimal string `D2FE28` becomes:

    110100101111111000101000
    VVVTTTAAAAABBBBBCCCCC
    

Below each bit is a label indicating its purpose:

*   The three bits labeled `V` (`110`) are the packet version, `6`.
*   The three bits labeled `T` (`100`) are the packet type ID, `4`, which means the packet is a literal value.
*   The five bits labeled `A` (`10111`) start with a `1` (not the last group, keep reading) and contain the first four bits of the number, `0111`.
*   The five bits labeled `B` (`11110`) start with a `1` (not the last group, keep reading) and contain four more bits of the number, `1110`.
*   The five bits labeled `C` (`00101`) start with a `0` (last group, end of packet) and contain the last four bits of the number, `0101`.
*   The three unlabeled `0` bits at the end are extra due to the hexadecimal representation and should be ignored.

So, this packet represents a literal value with binary representation `011111100101`, which is `2021` in decimal.

Every other type of packet (any packet with a type ID other than `4`) represent an _operator_ that performs some calculation on one or more sub-packets contained within. Right now, the specific operations aren't important; focus on parsing the hierarchy of sub-packets.

An operator packet contains one or more packets. To indicate which subsequent binary data represents its sub-packets, an operator packet can use one of two modes indicated by the bit immediately after the packet header; this is called the _length type ID_:

*   If the length type ID is `0`, then the next _15_ bits are a number that represents the _total length in bits_ of the sub-packets contained by this packet.
*   If the length type ID is `1`, then the next _11_ bits are a number that represents the _number of sub-packets immediately contained_ by this packet.

Finally, after the length type ID bit and the 15-bit or 11-bit field, the sub-packets appear.

For example, here is an operator packet (hexadecimal string `38006F45291200`) with length type ID `0` that contains two sub-packets:

    00111000000000000110111101000101001010010001001000000000
    VVVTTTILLLLLLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBBBBBB
    

*   The three bits labeled `V` (`001`) are the packet version, `1`.
*   The three bits labeled `T` (`110`) are the packet type ID, `6`, which means the packet is an operator.
*   The bit labeled `I` (`0`) is the length type ID, which indicates that the length is a 15-bit number representing the number of bits in the sub-packets.
*   The 15 bits labeled `L` (`000000000011011`) contain the length of the sub-packets in bits, `27`.
*   The 11 bits labeled `A` contain the first sub-packet, a literal value representing the number `10`.
*   The 16 bits labeled `B` contain the second sub-packet, a literal value representing the number `20`.

After reading 11 and 16 bits of sub-packet data, the total length indicated in `L` (27) is reached, and so parsing of this packet stops.

As another example, here is an operator packet (hexadecimal string `EE00D40C823060`) with length type ID `1` that contains three sub-packets:

    11101110000000001101010000001100100000100011000001100000
    VVVTTTILLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBCCCCCCCCCCC
    

*   The three bits labeled `V` (`111`) are the packet version, `7`.
*   The three bits labeled `T` (`011`) are the packet type ID, `3`, which means the packet is an operator.
*   The bit labeled `I` (`1`) is the length type ID, which indicates that the length is a 11-bit number representing the number of sub-packets.
*   The 11 bits labeled `L` (`00000000011`) contain the number of sub-packets, `3`.
*   The 11 bits labeled `A` contain the first sub-packet, a literal value representing the number `1`.
*   The 11 bits labeled `B` contain the second sub-packet, a literal value representing the number `2`.
*   The 11 bits labeled `C` contain the third sub-packet, a literal value representing the number `3`.

After reading 3 complete sub-packets, the number of sub-packets indicated in `L` (3) is reached, and so parsing of this packet stops.

For now, parse the hierarchy of the packets throughout the transmission and _add up all of the version numbers_.

Here are a few more examples of hexadecimal-encoded transmissions:

*   `8A004A801A8002F478` represents an operator packet (version 4) which contains an operator packet (version 1) which contains an operator packet (version 5) which contains a literal value (version 6); this packet has a version sum of _`16`_.
*   `620080001611562C8802118E34` represents an operator packet (version 3) which contains two sub-packets; each sub-packet is an operator packet that contains two literal values. This packet has a version sum of _`12`_.
*   `C0015000016115A2E0802F182340` has the same structure as the previous example, but the outermost packet uses a different length type ID. This packet has a version sum of _`23`_.
*   `A0016C880162017C3686B18A3D4780` is an operator packet that contains an operator packet that contains an operator packet that contains five literal values; it has a version sum of _`31`_.

Decode the structure of your hexadecimal-encoded BITS transmission; _what do you get if you add up the version numbers in all packets?_

### Part Two

Now that you have the structure of your transmission decoded, you can calculate the value of the expression it represents.

Literal values (type ID `4`) represent a single number as described above. The remaining type IDs are more interesting:

*   Packets with type ID `0` are _sum_ packets - their value is the sum of the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
*   Packets with type ID `1` are _product_ packets - their value is the result of multiplying together the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
*   Packets with type ID `2` are _minimum_ packets - their value is the minimum of the values of their sub-packets.
*   Packets with type ID `3` are _maximum_ packets - their value is the maximum of the values of their sub-packets.
*   Packets with type ID `5` are _greater than_ packets - their value is _1_ if the value of the first sub-packet is greater than the value of the second sub-packet; otherwise, their value is _0_. These packets always have exactly two sub-packets.
*   Packets with type ID `6` are _less than_ packets - their value is _1_ if the value of the first sub-packet is less than the value of the second sub-packet; otherwise, their value is _0_. These packets always have exactly two sub-packets.
*   Packets with type ID `7` are _equal to_ packets - their value is _1_ if the value of the first sub-packet is equal to the value of the second sub-packet; otherwise, their value is _0_. These packets always have exactly two sub-packets.

Using these rules, you can now work out the value of the outermost packet in your BITS transmission.

For example:

*   `C200B40A82` finds the sum of `1` and `2`, resulting in the value _`3`_.
*   `04005AC33890` finds the product of `6` and `9`, resulting in the value _`54`_.
*   `880086C3E88112` finds the minimum of `7`, `8`, and `9`, resulting in the value _`7`_.
*   `CE00C43D881120` finds the maximum of `7`, `8`, and `9`, resulting in the value _`9`_.
*   `D8005AC2A8F0` produces `1`, because `5` is less than `15`.
*   `F600BC2D8F` produces `0`, because `5` is not greater than `15`.
*   `9C005AC2F8F0` produces `0`, because `5` is not equal to `15`.
*   `9C0141080250320F1802104A08` produces `1`, because `1` + `3` = `2` \* `2`.

_What do you get if you evaluate the expression represented by your hexadecimal-encoded BITS transmission?_


In [1]:
import numpy as np
import copy
from time import perf_counter
from collections import Counter, deque
from functools import lru_cache
import numba

In [2]:
f = open("input.txt", "r")
rawstring = f.read()
f.close()

In [3]:
bytes = bytearray.fromhex(rawstring)
bytes_as_bits = ''.join(format(byte, '08b') for byte in bytes)
bytes_as_bits


'100000100000110101001010100000000001111011100000000001110010000000011001000011001010000000000101001000000001011010000010101000000000010010011000000000010100110000000100101110111011000000010001100001101100000001000000101000100000000011101100011001100000000001101001000000001100010001001000000000101011101000101000000000010000010000000010000110110011000000000111000010100100000000010110100110000000000001000100110010000000000010111000010010110101111100010011101111111111000000000111000010000001100000000000111111101001011111111101111110000011000001000000000110111111010010100110111000100011100110100000000010011100110011100010001011100101001111011100100101000010100111000001011100000000000100111010100011000000000111101000011111010001000000100011100110011000000000111111000100010010000010110100011000110010000000000100001001100001000001000101000110000011111100110000001111100100000000010111110111100000000000101111001100101001001011001011000001001101111010000110111001101110011111100101010000010000000

In [4]:
class Packet:
    def __init__(self, version, type_id,src_string) -> None:
        self.version = version
        self.type_id = type_id
        self.operator = self.type_id != 4
        self.src_string = src_string
        self.startSub = []
        self.sub = []

        if self.operator:
            self.l_type_id = int(src_string[6])

            if self.l_type_id == 0:
                self.sub_length = int(src_string[7:7+15], base=2)
                self.startSub.append(7+15)
                self.end = 7+15+self.sub_length
                while self.startSub[-1] != self.end:
                    self.parseSub(self.src_string[self.startSub[-1]:self.end])
                    self.startSub.append(self.startSub[-1]+self.sub[-1].end)
            else: 
                self.n_sub = int(src_string[7:7+11], base=2)
                self.startSub.append(7+11)
                while len(self.sub) != self.n_sub:
                    self.parseSub(self.src_string[self.startSub[-1]:])
                    self.startSub.append(self.startSub[-1]+self.sub[-1].end)
                self.end = self.startSub[-1]
            
        else:
            #literal
            temp = list(self.src_string[6:])
            self.len_lit = 5
            i=0
            while(temp[i] != '0'):
                self.len_lit += 5
                temp.pop(i)
                i += 4
            temp.pop(i)
            temp = temp[:i+4]
            self.literalVal = int("".join(temp),base=2)
            self.end = 6+self.len_lit
            self.val = self.literalVal

        if self.type_id == 0:
            self.val = sum([x.val for x in self.sub])
        elif self.type_id == 1:
            self.val = np.product([x.val for x in self.sub])
        elif self.type_id == 2:
            self.val = min([x.val for x in self.sub])
        elif self.type_id == 3:
            self.val = max([x.val for x in self.sub])
        elif self.type_id == 5:
            self.val = 1 if self.sub[0].val > self.sub[1].val else 0
        elif self.type_id == 6:
            self.val = 1 if self.sub[0].val < self.sub[1].val else 0
        elif self.type_id == 7:
            self.val = 1 if self.sub[0].val == self.sub[1].val else 0

        

    def summedVersion(self):
        return self.version + sum([x.summedVersion() for x in self.sub])

    def parseSub(self, src_string):
        self.sub.append(inter := Packet.fromString(src_string))
        #print(f"{inter.end=}")

    @staticmethod
    def fromString(src_string):
        version = int(src_string[0:3], base=2)
        type_id = int(src_string[3:6], base=2)
        return Packet(version, type_id, src_string)

In [5]:
outer = Packet.fromString(bytes_as_bits)

part 1:

In [6]:
outer.summedVersion()

974

part 2:

In [7]:
outer.val

180616437720